In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
import gc

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
# 레이블 인코딩

str_col = ['day_of_week','start_turn_restricted','end_turn_restricted']
for i in str_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: 
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])

In [4]:
y_train = train['target'] 
X_train = train.drop(['id','base_date','road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)


X_test = test.drop(['id','base_date', 'road_name', 'start_node_name', 'end_node_name','vehicle_restricted'], axis=1)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

(4701217, 17)
(4701217,)
(291241, 16)


In [5]:
X_train

,day_of_week,base_hour,lane_count,road_rating,multi_linked,connect_code,maximum_speed_limit,weight_restricted,height_restricted,road_type,start_latitude,start_longitude,start_turn_restricted,end_latitude,end_longitude,end_turn_restricted,target
0,1,17,1,106,0,0,60.0,32400.0,0.0,3,33.427747,126.662612,0,33.427749,126.662335,0,52.0
1,1,21,2,103,0,0,60.0,0.0,0.0,0,33.500730,126.529107,1,33.504811,126.526240,0,30.0
2,4,7,2,103,0,0,80.0,0.0,0.0,0,33.279145,126.368598,0,33.280072,126.362147,0,61.0
3,0,13,2,107,0,0,50.0,0.0,0.0,0,33.246081,126.567204,0,33.245565,126.566228,0,20.0
4,6,8,2,103,0,0,80.0,0.0,0.0,0,33.462214,126.326551,0,33.462677,126.330152,0,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4701212,1,16,1,107,0,0,50.0,0.0,0.0,0,33.422145,126.278125,0,33.420955,126.273750,0,20.0
4701213,1,2,2,107,0,0,80.0,43200.0,0.0,3,33.472505,126.424368,0,33.472525,126.424890,0,65.0
4701214,3,22,2,103,0,0,60.0,0.0,0.0,0,33.447183,126.912579,0,33.444121,126.912948,0,30.0
4701215,2,2,2,103,0,0,80.0,0.0,0.0,0,33.443596,126.431817,0,33.444996,126.433332,0,73.0


In [6]:
X_test

,day_of_week,base_hour,lane_count,road_rating,multi_linked,connect_code,maximum_speed_limit,weight_restricted,height_restricted,road_type,start_latitude,start_longitude,start_turn_restricted,end_latitude,end_longitude,end_turn_restricted
0,1,17,3,107,0,0,70.0,0.0,0.0,0,33.499427,126.541298,0,33.500772,126.543837,1
1,6,12,2,103,0,0,70.0,0.0,0.0,3,33.258507,126.427003,0,33.258119,126.415840,0
2,0,2,1,103,0,0,60.0,0.0,0.0,0,33.258960,126.476508,0,33.259206,126.474687,0
3,1,23,3,103,0,0,70.0,0.0,0.0,0,33.473494,126.545647,0,33.471061,126.545467,0
4,2,17,3,106,0,0,70.0,0.0,0.0,0,33.501477,126.569223,0,33.496863,126.581230,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
291236,5,5,1,103,0,0,80.0,0.0,0.0,0,33.472819,126.463030,0,33.470483,126.460846,0
291237,0,20,2,103,0,0,60.0,0.0,0.0,0,33.305359,126.598914,0,33.300796,126.600332,0
291238,0,11,1,107,0,0,30.0,0.0,0.0,0,33.493624,126.496769,1,33.497500,126.496946,1
291239,0,7,2,107,0,0,60.0,0.0,0.0,0,33.254782,126.507014,0,33.255659,126.507333,0


In [7]:
from pycaret.regression import *

In [8]:
X_train2 = X_train.loc[:800000, : ]

# 4백만개의 row를 한꺼번에 돌리니까 터지더라구요. 그래서 일부분만!

In [9]:
X_train2

,day_of_week,base_hour,lane_count,road_rating,multi_linked,connect_code,maximum_speed_limit,weight_restricted,height_restricted,road_type,start_latitude,start_longitude,start_turn_restricted,end_latitude,end_longitude,end_turn_restricted,target
0,1,17,1,106,0,0,60.0,32400.0,0.0,3,33.427747,126.662612,0,33.427749,126.662335,0,52.0
1,1,21,2,103,0,0,60.0,0.0,0.0,0,33.500730,126.529107,1,33.504811,126.526240,0,30.0
2,4,7,2,103,0,0,80.0,0.0,0.0,0,33.279145,126.368598,0,33.280072,126.362147,0,61.0
3,0,13,2,107,0,0,50.0,0.0,0.0,0,33.246081,126.567204,0,33.245565,126.566228,0,20.0
4,6,8,2,103,0,0,80.0,0.0,0.0,0,33.462214,126.326551,0,33.462677,126.330152,0,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799996,1,22,3,103,0,0,70.0,0.0,0.0,0,33.478257,126.468081,1,33.480679,126.472216,1,39.0
799997,4,13,2,107,0,0,80.0,0.0,0.0,0,33.472276,126.418443,0,33.472505,126.424368,0,69.0
799998,3,9,2,103,0,0,60.0,0.0,0.0,3,33.265065,126.372064,0,33.265140,126.372580,0,49.0
799999,2,17,1,106,0,0,60.0,0.0,0.0,0,33.529194,126.839578,0,33.496623,126.812152,0,45.0


In [20]:
X_train2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 800001 entries, 0 to 800000
Data columns (total 17 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   day_of_week            800001 non-null  int32  
 1   base_hour              800001 non-null  int64  
 2   lane_count             800001 non-null  int64  
 3   road_rating            800001 non-null  int64  
 4   multi_linked           800001 non-null  int64  
 5   connect_code           800001 non-null  int64  
 6   maximum_speed_limit    800001 non-null  float64
 7   weight_restricted      800001 non-null  float64
 8   height_restricted      800001 non-null  float64
 9   road_type              800001 non-null  int64  
 10  start_latitude         800001 non-null  float64
 11  start_longitude        800001 non-null  float64
 12  start_turn_restricted  800001 non-null  int32  
 13  end_latitude           800001 non-null  float64
 14  end_longitude          800001 non-nu

In [10]:
# 불러온 pycaret 셋업하기

# https://ideal-dominate.medium.com/pycaret-regression-%EA%B0%99%EC%9D%B4-%ED%95%B4%EB%B4%90%EC%9A%94-eaf0e36268

sup = setup(X_train2, target = 'target' , train_size = 0.8 )

,Description,Value
0,session_id,7080
1,Target,target
2,Original Data,"(800001, 17)"
3,Missing Values,False
4,Numeric Features,11
5,Categorical Features,5
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(640000, 18)"


In [11]:
models()

,Name,Reference,Turbo
ID,,,
lr,Linear Regression,sklearn.linear_model._base.LinearRegression,True
lasso,Lasso Regression,sklearn.linear_model._coordinate_descent.Lasso,True
ridge,Ridge Regression,sklearn.linear_model._ridge.Ridge,True
en,Elastic Net,sklearn.linear_model._coordinate_descent.Elast...,True
lar,Least Angle Regression,sklearn.linear_model._least_angle.Lars,True
llar,Lasso Least Angle Regression,sklearn.linear_model._least_angle.LassoLars,True
omp,Orthogonal Matching Pursuit,sklearn.linear_model._omp.OrthogonalMatchingPu...,True
br,Bayesian Ridge,sklearn.linear_model._bayes.BayesianRidge,True
ard,Automatic Relevance Determination,sklearn.linear_model._bayes.ARDRegression,False


In [23]:
# 모델 비교하기

compare_models()

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
xgboost,Extreme Gradient Boosting,4.0263,30.5585,5.5279,0.8798,0.1729,0.1265,39.7990
rf,Random Forest Regressor,3.9265,31.2857,5.5933,0.8769,0.1711,0.1198,133.8720
catboost,CatBoost Regressor,4.1128,31.8228,5.6411,0.8748,0.1756,0.1290,56.6360
et,Extra Trees Regressor,4.0169,33.0642,5.7501,0.8699,0.1761,0.1222,126.3300
dt,Decision Tree Regressor,4.0306,33.3732,5.7769,0.8687,0.1771,0.1226,2.5370
knn,K Neighbors Regressor,4.6382,42.0107,6.4815,0.8347,0.1938,0.1418,835.9500
lightgbm,Light Gradient Boosting Machine,4.9777,42.8562,6.5464,0.8314,0.2014,0.1570,2.6070
gbr,Gradient Boosting Regressor,6.5962,71.9525,8.4824,0.7169,0.2527,0.2108,51.5450
ada,AdaBoost Regressor,9.6228,138.7523,11.7765,0.4542,0.3148,0.2846,35.3430
br,Bayesian Ridge,9.8508,147.8071,12.1576,0.4185,0.3440,0.3106,0.5030


XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=100,
             n_jobs=-1, num_parallel_tree=1, objective='reg:squarederror',
             predictor='auto', random_state=4633, reg_alpha=0, ...)

In [24]:
# xgboost 나 rf로 하면 되겠다!

# Extra Trees Regressor 이나 Decision Tree Regressor도 써보면 좋겠다!



In [12]:
top1 = create_model('xgboost' , cross_valodation = False )

,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,4.2459,33.6331,5.7994,0.8664,0.1791,0.1325
1,4.2464,33.6339,5.7995,0.8672,0.1771,0.1314
2,4.3023,34.6382,5.8854,0.8640,0.1803,0.1339
3,4.2649,33.9684,5.8282,0.8670,0.1785,0.1327
4,4.2378,33.6308,5.7992,0.8680,0.1790,0.1328
5,4.3104,34.5106,5.8746,0.8645,0.1805,0.1344
6,4.3092,34.7027,5.8909,0.8630,0.1807,0.1340
7,4.2233,33.4355,5.7823,0.8685,0.1781,0.1318
8,4.3042,34.5814,5.8806,0.8652,0.1810,0.1344


In [13]:
top2 = create_model( 'rf' , cross_validation = False )

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,4.1739,34.8425,5.9028,0.863,0.1768,0.1265


In [ ]:
# 파라미터 튜닝

tuned_xgboost = tune_model(top1 , optimize = 'RMSE' , n_iter = 10)
tuned_rf = tune_model(top2 , optimize = 'RMSE' , n_iter = 10  )


,MAE,MSE,RMSE,R2,RMSLE,MAPE
Fold,,,,,,
0,3.9912,30.9326,5.5617,0.8772,0.1714,0.1237
1,3.9776,30.8441,5.5538,0.8783,0.1693,0.1224
2,3.9990,31.4472,5.6078,0.8765,0.1717,0.1239
3,3.9904,31.0474,5.5720,0.8784,0.1698,0.1232
4,3.9561,30.5549,5.5276,0.8800,0.1701,0.1233
5,3.9911,31.0359,5.5710,0.8782,0.1710,0.1238
6,4.0008,31.3468,5.5988,0.8762,0.1714,0.1237
7,3.9537,30.6276,5.5342,0.8796,0.1701,0.1228
8,3.9837,31.0608,5.5732,0.8790,0.1706,0.1232


IntProgress(value=0, description='Processing: ', max=7)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,05:23:08
Status,. . . . . . . . . . . . . . . . . .,Searching Hyperparameters
Estimator,. . . . . . . . . . . . . . . . . .,Random Forest Regressor


,Fold,MAE,MSE,RMSE,R2,RMSLE,MAPE


Fitting 10 folds for each of 10 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
